Use this to load Tensorflow modelzoo models from export to run inference and inspect predictions.

First some imports:

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import matplotlib
from matplotlib import patches, text, patheffects
import matplotlib.pyplot as plt
import io
import cv2
from src.image_tools import ImageTools
import sys
import random
import colorsys
import json

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#if len(physical_devices) > 0:
#    tf.config.experimental.set_memory_growth(physical_devices[0], True)
#from absl import app, flags, logging
#from absl.flags import FLAGS
import core.utils as utils
from core.config import cfg
from core.yolov4 import filter_boxes
from tensorflow.python.saved_model import tag_constants
from PIL import Image

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


Constants

In [2]:
###Not used
THRESHOLD = 0.1
YOLO = False
#MODEL_PATH = 'trained_models/faster_rcnn/saved_model'
MODEL_PATH = 'trained_models/efficientDet_d1/saved_model'
#MODEL_PATH = 'trained_models/yolov4_1024'
#MODEL_PATH = 'trained_models/faster_rcnn_5ksteps_dropout5/saved_model'
#IMAGE_PATH = "C:/Users/Sigurd/OneDriveMS/FYS-3741-MASTER/data/data_yoloformat/test/images"
#IMAGE_PATH = "C:/Users/sigur/OneDriveMS/FYS-3741-MASTER/data/data_yoloformat/test/images"
IMAGE_PATH = "C:/Users/Sigurd/OneDriveMS/FYS-3741-MASTER/Tensorflow/workspace/inference_tools/examples/test_im"
#IMAGE_PATH = "C:/Users/sigur/OneDriveMS/FYS-3741-MASTER/Tensorflow/workspace/inference_tools/examples/test_im"
PLOT = True
WRITE_TO_FILE = False
#RESULT_PATH = "New_labels/Frcnn_dropout.json"
RESULT_PATH = "New_labels/EffDetD1_nms.json"
#INPUT_SIZE = 1024


Load functions

In [3]:
def draw_bbox(image, bboxes, classes="Porpoise", show_label=True):
    num_classes = 1
    image_h, image_w, _ = image.shape
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    out_boxes, out_scores, out_classes, num_boxes = bboxes
    #print(num_boxes)
    for i in range(num_boxes):
        if int(out_classes[i]) < 0 or int(out_classes[i]) > num_classes: continue
        coor = out_boxes[i]
        coor[0] = int(coor[0] * image_h)
        coor[2] = int(coor[2] * image_h)
        coor[1] = int(coor[1] * image_w)
        coor[3] = int(coor[3] * image_w)
        #### fontscale = 0.5
        fontScale = 1
        score = out_scores[i]
        class_ind = int(out_classes[i])
        class_name = classes
        # check if class is in allowed classes

        bbox_color = colors[0]
        bbox_thick = int(0.6 * (image_h + image_w) / 600)
        c1, c2 = (coor[1], coor[0]), (coor[3], coor[2])
        cv2.rectangle(image, c1, c2, bbox_color, bbox_thick)
        
        if show_label:
            bbox_mess = '%s: %.2f' % ("Porpoise", score)
            t_size = cv2.getTextSize(bbox_mess, 0, fontScale, thickness=bbox_thick // 2)[0]
            c3 = (c1[0] + t_size[0], c1[1] - t_size[1] - 3)
            cv2.rectangle(image, c1, (np.float32(c3[0]), np.float32(c3[1])), bbox_color, -1) #filled

            cv2.putText(image, bbox_mess, (c1[0], np.float32(c1[1] - 2)), cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale, (0, 0, 0), bbox_thick // 2, lineType=cv2.LINE_AA)
    return image


def load_image_to_numpy_array(path):
    """
    Load a single image to numpy array
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(io.BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(im_height, im_width, 3).astype(np.uint8)

def convert_bb(bbox, img_shape):
    """
    Convert bounding box coordinates from (ymin, xmin, ymax, xmax) to absolute form (xmin, ymin, w, h)
    for use in evaluate_model2.ipynb
    """
    boxes = np.zeros(bbox.shape)


    boxes[:,0] = np.round((bbox[:,1])*img_shape[1],0)


    boxes[:,1] = np.round(bbox[:,0]*img_shape[0],0)
    boxes[:,2] = np.round(((bbox[:,3] - bbox[:,1])* img_shape[1]),0)

    boxes[:,3] = np.round(((bbox[:,2] - bbox[:,0]) * img_shape[0]),0)
    #print(boxes[:,3])

    return boxes

Load model from file:

In [4]:
start_time = time.time()
tf.keras.backend.clear_session()
#Loads model
detect_fn = tf.saved_model.load(MODEL_PATH)
end_time = time.time()
elapsed_time = end_time - start_time
print('Loaded model in: {} seconds'.format(elapsed_time))

Loaded model in: 129.7710840702057 seconds


Run inference on model. Change plot to True if plotting

In [5]:

#image_dir = 'C:/Users/Sigurd/OneDriveMS/FYS-3741-MASTER/data/data_yoloformat/test_images_for_programming/'
images = os.listdir(IMAGE_PATH)
#image_names = os.listdir(image_dir)
iou = 0.5
score = 0.1
#json_result = []
start_time = time.time()
print('Processing Images:')

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
#STRIDES, ANCHORS, NUM_CLASS, XYSCALE = utils.load_config(FLAGS)
input_size = 1024
#images = FLAGS.images
#print('output1: ', output)
#if output != None:
#    output = 'output/images/'+output
# load model
#print(images)
#print('output: ', output)
#saved_model_loaded = tf.saved_model.load('./models/yolov4_1024', tags=[tag_constants.SERVING])
#saved_model_loaded = model
# loop through images in list and run Yolov4 model on each
def main():
    json_result = []    
    for count, im_path in enumerate(images, 1):

        #if count != 2:
        #    continue
        sys.stdout.write('\r Image: {}, Completion: {}%'.format(count+1, round((count+1) / len(images)*100)))
        #print('count: ', count)
        #print('im_path: ', im_path)
        image_path = IMAGE_PATH+'/'+im_path

        #print(image_path)
        if YOLO:
            original_image = cv2.imread(image_path)
            
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

            image_data = cv2.resize(original_image, (input_size, input_size))
            image_data = image_data / 255.

        else:
            original_image = cv2.imread(image_path)
            
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

            #image_data = cv2.resize(original_image, (input_size, input_size))
            image_data = original_image
            #print(image_data.dtype)


        images_data = []
        for i in range(1):
            images_data.append(image_data)
        images_data = np.asarray(images_data).astype(np.uint8)

        batch_data = tf.constant(images_data)

        infer = detect_fn.signatures['serving_default']
        pred_bbox = infer(batch_data)

        boxes = pred_bbox['detection_boxes'][0]
        pred_conf = pred_bbox['detection_scores'][0]
        classes = pred_bbox['detection_classes'][0]


        valid_pred = tf.image.non_max_suppression(
            boxes = boxes,
            scores = pred_conf,
            max_output_size = 10,
            iou_threshold = iou,
            score_threshold = score
        )

        bbox = tf.gather(boxes, valid_pred).numpy()

                    #bbox-coords    #confidence                             #class prediction                   #valid detections   
        pred_bbox = [bbox, tf.gather(pred_conf, valid_pred).numpy(), tf.gather(classes, valid_pred).numpy(), np.array(bbox.shape[0])]

        if PLOT:
            image = draw_bbox(original_image, pred_bbox)

            #cv2.imshow(im_path, image)

            image = Image.fromarray(image.astype(np.uint8))
            

            image.show(title='im_path')
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()

        if WRITE_TO_FILE:
            if len(bbox)==0:
                bbox_abs = []
            else:
                bbox_abs = convert_bb(bbox, image_data.shape)


            for i in range(pred_bbox[3]):
                json_result.append({
                    "image_id": im_path,
                    "category_id": int(pred_bbox[2][i]),
                    "bbox": bbox_abs[i],
                    "score": pred_bbox[1][i]
                })
        sys.stdout.flush()



    end_time = time.time()
    print('\n','Process complete in {} seconds \n'.format(round(end_time - start_time, 1)))

    print('Writing to file...' )
    if WRITE_TO_FILE:

        #print(len(json_result))
        with open('{}'.format(RESULT_PATH), 'w') as outfile:
            outfile.write('[')
            outfile.write('\n')
            for i in range(len(json_result)):
                json.dump(str(json_result[i]), outfile, separators=(',', ':'))
                if i < len(json_result) - 1:
                    outfile.write(',\n')
                else:
                    outfile.write('\n')
            outfile.write(']')

    print('Writing to file complete.')

main()




Processing Images:
 Image: 2, Completion: 22%

C:\Users\Sigurd\AppData\Local\Temp\ipykernel_19604\944860231.py:31: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.rectangle(image, c1, c2, bbox_color, bbox_thick)
C:\Users\Sigurd\AppData\Local\Temp\ipykernel_19604\944860231.py:37: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.rectangle(image, c1, (np.float32(c3[0]), np.float32(c3[1])), bbox_color, -1) #filled
C:\Users\Sigurd\AppData\Local\Temp\ipykernel_19604\944860231.py:39: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.putText(image, bbox_mess, (c1[0], np.float32(c1[1] - 2)), cv2.FONT_HERSHEY_SIMPLEX,


 Image: 10, Completion: 111%
 Process complete in 20.9 seconds 

Writing to file...
Writing to file complete.
